In [42]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [48]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 16*5*5)
        x = self.fc_layers(x)
        return x

In [43]:
LeNet = nn.Sequential(
    nn.Conv2d(1,6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6,16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [44]:
X = torch.rand(size=(1,1,28,28), dtype=torch.float32)
for layer in LeNet:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2d output shape:	 torch.Size([1, 6, 28, 28])
Sigmoid output shape:	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape:	 torch.Size([1, 6, 14, 14])
Conv2d output shape:	 torch.Size([1, 16, 10, 10])
Sigmoid output shape:	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape:	 torch.Size([1, 16, 5, 5])
Flatten output shape:	 torch.Size([1, 400])
Linear output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


In [45]:
batch_size = 256
# Load MNIST dataset
train_dataset = MNIST(root="../data", train=True, download=False, transform=ToTensor())
test_dataset = MNIST(root="../data", train=False, download=False, transform=ToTensor())

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [46]:
device = torch.device("mps")
device

device(type='mps')

In [47]:
# Initialize the model and optimizer
model = LeNet.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100*correct/total:.2f}%')

Epoch [1/10], Step [100/1875], Loss: 2.3036
Epoch [1/10], Step [200/1875], Loss: 2.3276
Epoch [1/10], Step [300/1875], Loss: 2.3169
Epoch [1/10], Step [400/1875], Loss: 2.3001
Epoch [1/10], Step [500/1875], Loss: 2.2935
Epoch [1/10], Step [600/1875], Loss: 2.3305
Epoch [1/10], Step [700/1875], Loss: 2.2861
Epoch [1/10], Step [800/1875], Loss: 2.3038
Epoch [1/10], Step [900/1875], Loss: 2.2843
Epoch [1/10], Step [1000/1875], Loss: 2.2968
Epoch [1/10], Step [1100/1875], Loss: 2.3288
Epoch [1/10], Step [1200/1875], Loss: 2.2963
Epoch [1/10], Step [1300/1875], Loss: 2.2941
Epoch [1/10], Step [1400/1875], Loss: 2.3080
Epoch [1/10], Step [1500/1875], Loss: 2.3196
Epoch [1/10], Step [1600/1875], Loss: 2.3014
Epoch [1/10], Step [1700/1875], Loss: 2.3323
Epoch [1/10], Step [1800/1875], Loss: 2.3063
Epoch [2/10], Step [100/1875], Loss: 2.3164
Epoch [2/10], Step [200/1875], Loss: 2.2988
Epoch [2/10], Step [300/1875], Loss: 2.2834
Epoch [2/10], Step [400/1875], Loss: 2.2917
Epoch [2/10], Step [500

KeyboardInterrupt: 